This notebook is to exemplify the downloading from the database and cache into disk. 

In [1]:
#!ipcluster start -n 8 --engines=MPI --profile='mpi' # for parallel run: start the engines using terminal
from ipyparallel import Client
rc = Client(profile='mpi')

We first load the libraries necessary. We need to use pyfftw to define zeros_aligned arrays, and pyJHTDB for downloading. 

In [2]:
%%px
# Import the libraries

import os
import sys
import math
import numpy as np
import pyfftw as ft 
from mpi4py import MPI
import pyJHTDB
from pyJHTDB.dbinfo import isotropic1024coarse
from pyJHTDB import libJHTDB

Then we need to initialize the associated variables, including domain, pyJHTDB and MPI variables. 

In [3]:
%%px

comm = MPI.COMM_WORLD; rank = comm.Get_rank(); nproc = comm.Get_size()
if(rank==0):
    print("n_proc = "+str(nproc))
    print("rank = "+str(rank))

# pyJHTDB parameters
    
Nx = isotropic1024coarse['nx']; Ny = isotropic1024coarse['ny']; Nz = isotropic1024coarse['nz']
Lx = isotropic1024coarse['lx']; Ly = isotropic1024coarse['ly']; Lz = isotropic1024coarse['lz']
auth_key = "com.gmail.jhelsas-b854269a"
 
# Computational Domain

nx=Nx//nproc; ny=Ny; nz=Nz
nek=int(math.sqrt(2.0)/3*Nx)
time = 0.0

chkSz = 32
slabs = nx//chkSz

[stdout:0] 
n_proc = 1
rank = 0
[stdout:1] 
n_proc = 1
rank = 0
[stdout:2] 
n_proc = 1
rank = 0
[stdout:3] 
n_proc = 1
rank = 0
[stdout:4] 
n_proc = 1
rank = 0
[stdout:5] 
n_proc = 1
rank = 0
[stdout:6] 
n_proc = 1
rank = 0
[stdout:7] 
n_proc = 1
rank = 0


Here is the data loading part. It can be chosen to be one of two ways. In case there is a previously cached data in disk, it can be read directly without the need to download from the database. In this case, we supose that the data is stored in numpy array binary data, ".npz". Because of the way pyFFTW works, each component must be stored separately, therefore we initialize vx, vy and vz arrays as appropiate for the FFTW work, which is aligned arrays. Typical loading time from disk range from 20 to 60 seconds. 

In case the data has not been previously cached, it is necessary to download from the database. Due to constraints in the workings of the database itself, there is a maximum size that can be queried on a single function call, consequently, it is necessary to break the download into several queries, which can be described as follows: For a whole $1024^3$ download, each slab contained in a process is of size $128 \times 1024 \times 1024$, and each query is of size $32 \times 1024 \times 1024$, the data is stored temporarely in a list, then concatenated and than reshaped (via np.transpose) to be properly assigned to the vx, vy and vz vectors. As a precaution, the data is cached as soon as the final vectors are calculated. This notebook only calculates a single time-step, but it can be readly generalized to be used with a list of timesteps instead. Typical download times range from 170 to 250 seconds, inside sciserver.

In [7]:
%%px

vx = ft.zeros_aligned((nx,ny,nz), dtype='float32')
vy = ft.zeros_aligned((nx,ny,nz), dtype='float32')
vz = ft.zeros_aligned((nx,ny,nz), dtype='float32')

folder = "/home/idies/workspace/scratch"
filename = "check-isotropic1024coarse-"+str(rank)+"-(t="+str(time)+")"+".npz"
file = folder + "/" + filename

# Populate velocity field from the Database

if(rank==0):
    print("Starting the loading process")

##########################################
load_from_file = False

if(load_from_file):
    comm.Barrier(); t1=MPI.Wtime()
    content = np.load(file)
    if(int(content['nproc'])!=nproc):
        print("Unmatched number of processes. Must first pre-process to adequate number of process")
    vx = content['vx']
    vy = content['vy']
    vz = content['vz']
    comm.Barrier(); t2=MPI.Wtime()
    if(rank==0):
        print("Finished loading")
        sys.stdout.write('Load from disk: {0:.2f} seconds\n'.format(t2-t1))
else:
    comm.Barrier(); t1=MPI.Wtime()
    lJHTDB = libJHTDB(auth_key)
    lJHTDB.initialize()
    ud = []
    for k in range(slabs):
        if(rank==0):
            print(k)
        start = np.array([rank*nx+k*chkSz, 0, 0],dtype=np.int)
        width = np.array([chkSz,ny,nz],dtype=np.int)
        #start = np.array([ 0, 0, rank*nx+k*chkSz],dtype=np.int)
        ud.append(lJHTDB.getRawData(time,start,width, 
                                    data_set = 'isotropic1024coarse',
                                    getFunction = 'Velocity') )
    
    lJHTDB.finalize()
    comm.Barrier(); t2=MPI.Wtime()
    if(rank==0):
        print("Finished loading")
        sys.stdout.write('Load field from database: {0:.2f} seconds\n'.format(t2-t1))
    
    u = np.concatenate(ud,axis=2)
    comm.Barrier(); t1=MPI.Wtime()
    if(rank==0):
        sys.stdout.write('Concatenate: {0:.2f} seconds\n'.format(t1-t2))
    
    rsh = np.transpose(u,(2,1,0,3))
    comm.Barrier(); t2=MPI.Wtime()
    if(rank==0):
        sys.stdout.write('Transpose: {0:.2f} seconds\n'.format(t2-t1))
    ##########################################
    
    vx[:,:,:] = rsh[:,:,:,0]
    vy[:,:,:] = rsh[:,:,:,1]
    vz[:,:,:] = rsh[:,:,:,2]
    comm.Barrier(); t1=MPI.Wtime()
    if(rank==0):
        sys.stdout.write('Splitting: {0:.2f} seconds\n'.format(t1-t2))
        
if(rank==0):
    print("vx shape = "+str(vx.shape))

[stdout:0] 
Starting the loading process
0
1
2
3
Finished loading
Load field from database: 255.99 seconds
Concatenate: 45.22 seconds
Transpose: 0.03 seconds
Splitting: 82.94 seconds
vx shape = (128, 1024, 1024)


Once downloaded, we can cache the data on disk, as being done below. The tipical times required are pretty long, around 900-1100 seconds, which amounts to 15 to 20 minutes. 

In [6]:
%%px

folder = "/home/idies/workspace/scratch"
filename = "check-isotropic1024coarse-"+str(rank)+"-(t="+str(time)+")"+".npz"
file = folder + "/" + filename

comm.Barrier(); t1=MPI.Wtime()
np.savez(file,vx=vx,vy=vy,vz=vz,nproc=nproc)
comm.Barrier(); t2=MPI.Wtime()
if(rank==0):
    sys.stdout.write('Caching the data: {0:.2f} seconds\n'.format(t2-t1))

[stdout:0] Caching the data: 917.06 seconds
